# MNIST with Convolutional Neural Network (CNN)

## Assignment

MNIST is a dataset of handwritten digits. Below is the implementation of a relatively simple CNN that predicts the digit based on the image. The goals for this week are to:

1. Continue researching project/assemble papers you find.
2. Read through this notebook and try and gain an understanding. If you have any questions, message in the Discord.
3. Create your own notebook and implement a CNN. Try different hyperparameters, architectures, optimizers and see how it impacts the accuracy. Experiment to get a better understanding of how things work.

Again, if you have any questions, make sure to message the Discord chat and I will try to respond ASAP. I'm not sure if this is too much work for 1 week, so try to make as much progress as possible and bring your questions/comments to the next meeting. Also tell me if you think this is useful/worthless as I can do more/less of these. I'm hoping this will give everyone a basic understanding moving forward.

## Resources

Really good notes if you want to get a better understanding of the topics (http://cs231n.github.io/).

## Import Libraries

In [1]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

## Define Network

In [2]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        super(tutorial_model, self).__init__()
        self.convolution1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)     # Convolution Layer 1
        self.convolution2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)    # Convolution Layer 2
        
        self.conv_dropout = nn.Dropout()                                                 # Dropout Layer
        
        self.fc1 = nn.Linear(in_features=320, out_features=50)                           # Fully Connected Layer 1
        self.fc2 = nn.Linear(in_features=50, out_features=10)                            # Fully Connected Layer 2
        
        self.pool = nn.MaxPool2d(2, 2)                                                   # Max Pooling Layer
        
    def forward(self, x):
        """ Chain all layers together """
        x = self.pool(F.relu(self.convolution1(x)))    # input_x -> Convolution 1 -> RELU -> Pooling -> x_1
        x = self.conv_dropout(x)                       # x_1 -> Dropout -> x_2
        x = self.pool(F.relu(self.convolution2(x)))    # x_2 -> Convolution 2 -> RELU -> Pooling -> x_3
        
        x = x.view(-1, 320)                            # Collapse 3D tensor to 1D
        
        x = self.fc1(x)                                # x_3 -> Fully Connected 1 -> x_4                    
        y = self.fc2(x)                                # x_4 -> Fully Connected 2 -> y
        return y

### Architecture

The architecture I implemented is a very simple (common introductory). Here is a visualization.

![](http://playagricola.com/Kaggle/extract.png)

For more information about this architecture and other potential architectures, check out this link (https://www.kaggle.com/cdeotte/how-to-choose-cnn-architecture-mnist).

### Convolution

Each channel in a convolution represents a different kernel convolving over the input image. The "learning" in a CNN is primarily about changing the weights of these kernels. Here are some visualizations of convolutions.

![](https://www.learnopencv.com/wp-content/uploads/2017/11/convolution-example-matrix.gif)


![](https://colah.github.io/posts/2014-07-Understanding-Convolutions/img/RiverTrain-ImageConvDiagram.png)


This is the equation for determining the dimensions of an image after a convolution. It is useful when doing the math for kernel sizes when developing a network.

![](https://miro.medium.com/max/660/1*D47ER7IArwPv69k3O_1nqQ.png)

For more information/visualizations, check out this link (https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1).

### Max Pooling

Max Pooling is like a convolution where the stride = kernel size and the value is the max value. Here is a visualization.

![](https://miro.medium.com/max/2344/1*ReZNSf_Yr7Q1nqegGirsMQ@2x.png)

### RELU

RELU is one of the many activation units used to introduce non-linearity to a network. Non-linearity is what allows a network to model complex functions. RELU is one of the most commonly used activation units. Here is a graph of RELU.

![](https://miro.medium.com/max/357/1*oePAhrm74RNnNEolprmTaQ.png)

For more information about activation uints, check out this link (https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0).

### Dropout

Dropout is used in order to combat overfitting. Essentially, you randomly choose to ignore certain nodes, and by doing so, you help avoid overfitting. For more information, checkout this link (https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/dropout_layer.html).

### Fully Connected Layer

It's difficult to explain a Fully connected Layer in a couple lines. A lot of the concepts related to FC layers are important throughought ML. I would encourage you to do your own research to understand the topic. Some possible sources (https://towardsdatascience.com/understanding-neural-networks-19020b758230, https://www.youtube.com/watch?v=aircAruvnKk).

To just understand the code, we use Fully Connevted layers in order to convert the features calculated through convolutions into a classification. That is, we take the 4x4x20 matrix produced by the last convolution, flatten it into a vector of dimension 320x1, and then feed it into two fully connected layers, which output 10 values. We can then feed these 10 values to a softmax in order to calculate the probability the input image was a specfic digit.

## Set Hyperparameters

In [3]:
num_epochs = 3           # Number of epochs for training
batch_size = 64          # Batch Size for training/testing
learning_rate = 0.01     # Learning Rate for optimizer
momentum = 0.5           # Momentum for optimizer
dim = 28                 # Dimensions of MNIST

## Setup Data Loader

In [4]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize((0.5,), (0.5,)),             # Normalize grayscale data
])

In [5]:
# Download training data
trainset = torchvision.datasets.MNIST(root='./files', train=True, download=True, transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=8)

# Download testing Data
testset = torchvision.datasets.MNIST(root='./files', train=False, download=False, transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=8)

### MNIST Dataset
![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2016/05/Examples-from-the-MNIST-dataset.png)

## Initialize Model/Optimizer

In [6]:
model = tutorial_model()                                               # Initialize previously defined model
criterion = nn.CrossEntropyLoss()                                      # Definie loss function (Cross Entropy Loss)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)     # Initialize Optimizer (ADAM)
model.train();                                                         # Set model to training (updating weights)

### Cross Entropy Loss

This is the mathematical formulation of cross entropy loss.

![](https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/35_blog_image_38.png)

This provides us a way of measuring the accuracy of our model. For more information about loss functions in general, check out this link (https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html). For more information about cross entropy loss specifically, check out this link (https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a).

### ADAM

This is the mathematical formulation of the ADAM Optimizer. It is one of the more commonly used optimizers.

![](https://i.stack.imgur.com/08VZN.png)

Ultimately, most of machine learning is basically an optimization problem. You are trying to minimize the loss functions. Most optimization methods (including ADAM) revolve around calculating the gradient of the loss function and then taking a step to update the weights. For more information about ADAM, check out this link (https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/). 

## Train Model

In [ ]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        output = model(X_train_batch)                           # Forward pass through network
        curr_loss = criterion(output, Y_train_batch)            # Calculate loss of predictions
        loss.append(curr_loss.item())                           # Store loss

        optimizer.zero_grad()                                   # Clear last calculation
        curr_loss.backward()                                    # Calculate gradient based on loss
        optimizer.step()                                        # Update model weights

        _, predicted = torch.max(output.data, 1)                # Extract model predictions
        correct = (predicted == Y_train_batch).sum().item()     # Calculate number of correct predictions
        accuracy.append(correct/Y_train_batch.size(0))          # Calculate/store accuracy
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/3, Step: 1/938, Loss: 2.3555333614349365, Accuracy: 7.8125%
Epoch: 1/3, Step: 101/938, Loss: 0.38940656185150146, Accuracy: 92.1875%
Epoch: 1/3, Step: 201/938, Loss: 0.19838890433311462, Accuracy: 95.3125%
Epoch: 1/3, Step: 301/938, Loss: 0.31205636262893677, Accuracy: 90.625%
Epoch: 1/3, Step: 401/938, Loss: 0.13935035467147827, Accuracy: 98.4375%
Epoch: 1/3, Step: 501/938, Loss: 0.10574089735746384, Accuracy: 96.875%
Epoch: 1/3, Step: 601/938, Loss: 0.16353940963745117, Accuracy: 93.75%
Epoch: 1/3, Step: 701/938, Loss: 0.24433059990406036, Accuracy: 90.625%
Epoch: 1/3, Step: 801/938, Loss: 0.2317436933517456, Accuracy: 95.3125%
Epoch: 1/3, Step: 901/938, Loss: 0.2418145388364792, Accuracy: 92.1875%
Epoch: 2/3, Step: 1/938, Loss: 0.5233689546585083, Accuracy: 81.25%
Epoch: 2/3, Step: 101/938, Loss: 0.13728179037570953, Accuracy: 93.75%
Epoch: 2/3, Step: 201/938, Loss: 0.1206330731511116, Accuracy: 93.75%
Epoch: 2/3, Step: 301/938, Loss: 0.05145410820841789, Accuracy: 96.875%


## Test Model

In [8]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        output = model(X_test_batch)                            # Forward pass through network
        _, predicted = torch.max(output.data, 1)                # Extract prediction
        total += Y_test_batch.size(0)                           # Update total number of sample
        correct += (predicted == Y_test_batch).sum().item()     # Update number of correct predictions

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 96.41999999999999%
